In [166]:
from os.path import join, dirname
import math
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import SVD
# SVD = singular value decomposition

In [1]:
from moviebox.recommender import recommender

In [2]:
movieId = 32

In [3]:
movieId

32

In [4]:
recommendationsNumber = 10

In [10]:
showPlots = False

In [11]:
interactive = True

In [12]:
recommender(
    movieID=movieId,
    recommendationsNumber=recommendationsNumber,
    showPlots=showPlots,
    interactive=interactive
)

✔ Imported Data		0.2s
✔ Split Data		0.0s
✔ Trained Engine	6.0s
✔ Found Similarities	1.3s
❯ Given Movie
• Title: Zindagi - ID: 32
  Categories: Bollywood

★ Top 10 Recommendations
• Title: Kedi - ID: 3083
  Categories: Action

• Title: Bawarchi - ID: 3404
  Categories: Musical, Bollywood, World cinema

• Title: I Love You - ID: 926
  Categories: Romance Film, Drama

• Title: Vaastav: The Reality - ID: 3500
  Categories: Musical, World cinema, Bollywood

• Title: Lajja - ID: 4621
  Categories: Musical, Drama, Bollywood, World cinema

• Title: Baana Kaathadi - ID: 976
  Categories: Romance Film

• Title: Chetna - ID: 3455
  Categories: Action

• Title: Charas - ID: 1188
  Categories: Thriller, Action

• Title: Aaptamitra - ID: 2123
  Categories: Thriller, Horror, World cinema

• Title: Veera Bahu - ID: 3023
  Categories: Action



0

In [13]:
recommender

<function moviebox.recommender.recommender(movieID=2874, recommendationsNumber=3, showPlots=False, interactive=False)>

In [32]:
csvPath = pd.read_csv('/home/rahul/Workspace/spograde-recommendation-engine/moviebox/moviebox/dataset/movies.csv')

In [169]:
csvPath

,ID,Title,Category,Plot
0,0,Ghosts of Mars,"""Thriller"", ""Science Fiction"", ""Horror"", ""A...","Set in the second half of the 22nd century, th..."
1,1,White Of The Eye,"""Thriller"", ""Erotic thriller"", ""Psychologica...",A series of murders of rich young women throug...
2,2,A Woman in Flames,"""Drama""","Eva, an upper class housewife, becomes frustra..."
3,3,The Sorcerer's Apprentice,"""Family Film"", ""Fantasy"", ""Adventure"", ""Wor...","Every hundred years, the evil Morgana returns..."
4,4,Little city,"""Romantic comedy"", ""Ensemble Film"", ""Comedy-...","Adam, a San Francisco-based artist who works a..."
...,...,...,...,...
4995,4995,Morgan and Destiny's Eleventeenth Date: The Ze...,"""Short Film"", ""Romance Film"", ""Animation""",Morgan and Destiny pay a visit to an ornate ...
4996,4996,Indradhanura Chhai,"""Drama""",The film looks at the lives of three women liv...
4997,4997,Barfuß bis zum Hals,"""Comedy film""",A nudist site in eastern Germany is sold to Di...
4998,4998,Donovan's Reef,"""Romantic comedy"", ""Action/Adventure"", ""Come...","The film begins with Thomas ""Boats"" Gilhooley ..."


In [217]:
#values => return queryset containing dictionaries
#flatten() => json nested form
#tolist() => return the list of values
titles = csvPath[['Title']].values.flatten().tolist()

In [218]:
categories = csvPath[['Category']].values.flatten().tolist()

In [219]:
plot = csvPath[['Plot']].values.flatten().tolist()

In [249]:
dataset = {
    'titles': titles,
    'categories': categories,
    'plots': plot
}

In [221]:
def validateMovieID(movieId):
    if not (isinstance(movieId, int) and movieId >= 0 and movieId <= 4999):
        print('Invalid value for movieID {}'.format(movieId))
    return 0

In [222]:
validateMovieID(9000)

Invalid value for movieID 9000


0

In [223]:
{
    'title': dataset['titles'][32],
    'category': dataset['categories'][32]
}

{'title': 'Zindagi', 'category': '"Bollywood"'}

In [224]:
list_zip = []
for i in range(20):
    list={
        'title': dataset['titles'][i],
        'category': dataset['categories'][i]
    }
    list_zip.append(list)

In [225]:
TfidfVectorizer

sklearn.feature_extraction.text.TfidfVectorizer

In [226]:
vectorizer = TfidfVectorizer(
    lowercase=True,
    min_df=3,
    max_df=0.9,
    ngram_range=(1,2),
    stop_words='english'
)

In [239]:
plotsTFIDF = vectorizer.fit_transform(dataset['categories'])

In [240]:
cosineSimilarities = linear_kernel(plotsTFIDF, plotsTFIDF)

In [241]:
cosineSimilarities[32]

array([0., 0., 0., ..., 0., 0., 0.])

In [242]:
scores = enumerate(cosineSimilarities[32])

In [243]:
sortedScores = sorted(scores, key=lambda x: x[1], reverse=True)

In [244]:
scores

In [245]:
#5 => no. of recommendations
movie_Recommendations = sortedScores[1:5+1]

In [246]:

for movie in movie_Recommendations:
    a = {
        'movie_id': movie[0],
        'title': dataset['titles'][movie[0]],
        'category': dataset['categories'][movie[0]],
        'matching': math.ceil(movie[1]*100)
    }
    print(a)

{'movie_id': 1076, 'title': 'Meera', 'category': '"Bollywood"', 'matching': 100}
{'movie_id': 1397, 'title': 'Baseraa', 'category': '"Bollywood"', 'matching': 100}
{'movie_id': 1992, 'title': 'Yehi Hai Zindagi', 'category': '"Bollywood"', 'matching': 100}
{'movie_id': 2527, 'title': 'Jeevan Dhaara', 'category': '"Bollywood"', 'matching': 100}
{'movie_id': 3219, 'title': 'Dilwale', 'category': '"Bollywood"', 'matching': 100}
